# Introduction

This tutorial shows how to use classic methods for classification of terpenes.

The terpenes data is a subset of the [COCONUT](https://coconut.naturalproducts.net) dataset (version March 2021), which is obtained by filtering COCONUT's ``chemicalSuperClass`` column to include only ``Lipids and lipid-like molecules``.

# Setup

In [1]:
import pandas as pd

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

import napr
from napr.data import load_terpene
from napr.apps import Terpene
from napr.evaluation import eval_classification
from napr.utils import split_train_test, label_encode
from napr.hyperopt import find_best_models

RANDOM_STATE = 777

napr.__version__

'0.1.5'

# Data

In [2]:
# Download and load the terpene dataset.
# Note: the dataset, 'terpene-21.3.bz2', is saved by default to the current
# directory.
data = load_terpene(download=True)  # load_terpene() if downloaded already

data.head()

,_id,coconut_id,contains_sugar,heavy_atom_number,name,molecular_formula,molecular_weight,textTaxa,npl_noh_score,npl_score,...,weinerPathNumber,weinerPolarityNumber,zagrebIndex,topoPSA,tpsaEfficiency,iupac_name,chemicalClass,chemicalSubClass,chemicalSuperClass,directParentClassification
3,5f961a9bae0c19564532b966,CNP0330764,0,30,"10-hydroxy-5,9-dimethyl-15-[(3-methylbut-2-eno...",C25H36O5,416.551289,"[""plants"",""Oreoherzogia fallax"",""Ichthyothere ...",2.837158,2.158055,...,2090,69,176,83.83,0.201390,"10-hydroxy-5,9-dimethyl-15-[(3-methylbut-2-eno...",Prenol lipids,Diterpenoids,Lipids and lipid-like molecules,Kaurane diterpenoids
7,5f961a9bae0c19564532b96a,CNP0115481,0,32,"1,6,6,9a,11a-pentamethyl-1-(6-methylhepta-3,5-...",C30H48O2,440.702043,"[""notax""]",3.937131,2.325869,...,2710,74,188,40.46,0.091878,"1,6,6,9a,11a-pentamethyl-1-(6-methylhepta-3,5-...",Steroids and steroid derivatives,Cholestane steroids,Lipids and lipid-like molecules,Cholesterols and derivatives
10,5f961a9bae0c19564532b96d,CNP0151033,1,47,"7-[(acetyloxy)methyl]-4-({[4,5-dihydroxy-6-(hy...",C32H42O15,666.668134,"[""notax""]",3.253205,2.376088,...,8795,79,244,227.97,0.342168,"7-[(acetyloxy)methyl]-4-({[4,5-dihydroxy-6-(hy...",Prenol lipids,Terpene glycosides,Lipids and lipid-like molecules,Terpene glycosides
25,5f961a9cae0c19564532b97c,CNP0298418,0,54,"19-hydroxy-8,17-bis(hydroxymethyl)-1,2,8,15,17...",C41H64N2O11,760.955241,"[""plants""]",3.102837,2.178031,...,9906,134,328,218.27,0.287027,"19-hydroxy-8,17-bis(hydroxymethyl)-1,2,8,15,17...",Prenol lipids,Triterpenoids,Lipids and lipid-like molecules,Triterpenoids
29,5f961a9cae0c19564532b980,CNP0224557,0,47,"1-(acetyloxy)-1-[5-(acetyloxy)-4-{4,5,10-trihy...",C36H52O11,660.792866,"[""notax""]",3.742450,2.495217,...,7366,102,272,165.89,0.251215,"1-(acetyloxy)-1-[5-(acetyloxy)-4-{4,5,10-trihy...",Prenol lipids,Triterpenoids,Lipids and lipid-like molecules,Limonoids


## Preprocessing

In [3]:
terpene = Terpene(data=data)

# Data cleansing and feature engineering
terpene.preprocess()

terpene.data.head()

Data preprocessing finished in 0h:00m:1s.


,contains_sugar,heavy_atom_number,molecular_weight,npl_noh_score,npl_score,npl_sugar_score,number_of_carbons,number_of_nitrogens,number_of_oxygens,max_number_of_rings,...,bcutDescriptor_1,bcutDescriptor_2,bcutDescriptor_3,bcutDescriptor_4,bcutDescriptor_5,textTaxa_plants,textTaxa_marine,textTaxa_bacteria,textTaxa_fungi,chemicalSubClass
42937,0.0,0.425403,0.419676,-0.869772,-0.498512,-0.066886,0.786493,-0.390276,-0.361147,-0.543140,...,-0.245389,-0.449942,0.240168,-0.357102,-1.971594,0.0,0.0,0.0,0.0,Diradylglycerols
344415,0.0,-1.137152,-1.107608,0.928754,0.540304,0.874951,-1.296010,-0.390276,-0.509048,-0.492037,...,-0.245538,-0.298977,0.193125,-0.658767,-1.108608,0.0,0.0,0.0,0.0,Monoterpenoids
288712,0.0,-0.111725,-0.119538,0.306277,0.451287,0.797333,-0.153992,-0.390276,0.082555,-0.032113,...,-0.244959,-0.518734,0.227001,-0.618832,0.874734,1.0,0.0,0.0,0.0,Oxosteroids
317805,0.0,-0.209385,-0.234849,1.595586,1.186972,1.378648,0.047540,-0.390276,-0.656948,0.172298,...,-0.245679,-0.368269,0.190523,-0.493579,0.877609,1.0,0.0,0.0,0.0,Triterpenoids
162280,0.0,1.353170,1.372408,0.918086,0.844786,-0.968531,1.055203,-0.390276,1.857362,0.325606,...,-0.243217,-0.471545,0.567530,-0.384356,0.967057,0.0,0.0,0.0,0.0,Terpene glycosides


## Filtering and encoding the data

In [4]:
# Chemical subclasses of interest
selected_classes = [
    "Triterpenoids",
    "Diterpenoids",
    "Sesquiterpenoids",
    "Terpene lactones",
    "Terpene glycosides",
    "Monoterpenoids",
]

X_train, X_test, y_train, y_test = split_train_test(
    data=terpene.data,
    target="chemicalSubClass",
    selected_classes=selected_classes,
    test_size=0.2,
    random_state=RANDOM_STATE,
)

y_train_encoded, y_test_encoded, y_labels = label_encode(y_train, y_test)

# Classification using classic methods

Here, we benchmark several classic models, with their default parameters.

In [5]:
classifiers_classic = {
    "kNN": KNeighborsClassifier(),
    "Random Forest": RandomForestClassifier(class_weight="balanced_subsample"),
    "Naive Bayes": GaussianNB(),
    "RBF SVM": SVC(class_weight="balanced"),
    "Linear SVM": SVC(kernel="linear", class_weight="balanced"),
    "XGBoost": XGBClassifier(),
}

# Note: XGBClassifier only accepts encoded labels (y parameter)
bench_classic = eval_classification(
    classifiers_classic,
    X=X_train,
    y=y_train_encoded,
    test_size=0.2,
    random_state=RANDOM_STATE,
    n_jobs=-1,
)

pd.DataFrame(bench_classic)

100%|██████████| 6/6 [01:07<00:00, 11.22s/it, method=XGBoost]      


,estimator,time,accuracy,precision,recall,f1,conf_mat
0,kNN,0.007540,0.860735,0.866789,0.860735,0.860453,"[[1437.2700467457682, 12.578209277238404, 39.4..."
1,Random Forest,1.332742,0.969628,0.970320,0.969628,0.969584,"[[1538.7342682488168, 0.0, 4.192736425746134, ..."
2,Naive Bayes,0.027224,0.491425,0.503830,0.491425,0.458475,"[[317.80942107155533, 637.2959367134159, 50.31..."
3,RBF SVM,17.320930,0.843755,0.847854,0.843755,0.842392,"[[1306.456670262499, 94.75584322186275, 42.765..."
4,Linear SVM,26.742417,0.763818,0.771304,0.763818,0.755932,"[[1121.9762675296836, 241.50161812297648, 50.3..."
5,XGBoost,7.317692,0.999692,0.999693,0.999692,0.999692,"[[1554.6666666666508, 0.0, 0.0, 0.0, 0.0, 0.0]..."


XGBoost provides the best results.

Although the metrics are very close to 1, meaning it is functioning very good, we perform hyperparameter optimization to show how it works.

## Hyperparameter optimization

In [6]:
# Define the search space
# Note: you can change the space by changing the parameters of the classifier.
# 'hp' is an object of the 'keras_tuner.HyperParameters' class
def build_hypermodel(hp):
    search_space = {
        "n_estimators": hp.Choice(
            name="n_estimators", values=[50, 100, 200, 500, 1000], default=100
        ),
        "max_depth": hp.Int(
            name="max_depth", min_value=3, max_value=21, step=2
        ),
        "min_child_weight": hp.Int(
            name="min_child_weight", min_value=1, max_value=10
        ),
        "gamma": hp.Float(
            name="gamma", min_value=0.0, max_value=0.50, step=0.1
        ),
        "subsample": hp.Float(
            name="subsample", min_value=0.25, max_value=1, step=0.25
        ),
        "colsample_bytree": hp.Float(
            name="colsample_bytree", min_value=0.25, max_value=1, step=0.25
        ),
        "learning_rate": hp.Float(
            name="learning_rate", min_value=1e-4, max_value=1e-1, sampling="log"
        ),
        "random_state": hp.Fixed(name="random_state", value=RANDOM_STATE),
        "n_jobs": hp.Fixed(name="n_jobs", value=-1),
    }

    return XGBClassifier(**search_space)


# This saves the results of the hyperparameter search to the "tuner" folder in
# the current directory.
best_model_classic = find_best_models(
    X=X_train,
    y=y_train_encoded,
    max_trials=5,
    cv=5,
    hypermodel=build_hypermodel,
    project_name="tuner",
    overwrite=True,
    random_state=RANDOM_STATE,
)

best_model_classic

Trial 5 Complete [00h 02m 29s]
score: 0.9437634124673329

Best score So Far: 0.9996686854708614
Total elapsed time: 00h 17m 46s
INFO:tensorflow:Oracle triggered exit


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.5,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0.30000000000000004, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.01808265479567698,
              max_bin=256, max_cat_to_onehot=4, max_delta_step=0, max_depth=15,
              max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=500, n_jobs=-1,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=777, reg_alpha=0, ...)

## Evaluation

Now that we got the best model, we can evaluate it.

In [7]:
# Validation
results_val_classic = eval_classification(
    best_model_classic,
    X=X_train,
    y=y_train_encoded,
    test_size=0.2,
    random_state=RANDOM_STATE,
    n_jobs=-1,
)

pd.DataFrame(results_val_classic)

100%|██████████| 1/1 [01:00<00:00, 60.45s/it, method=XGBClassifier]


,estimator,time,accuracy,precision,recall,f1,conf_mat
0,XGBClassifier,60.341497,0.999598,0.999598,0.999598,0.999598,"[[1554.6666666666508, 0.0, 0.0, 0.0, 0.0, 0.0]..."


In [8]:
# Test
results_test_classic = eval_classification(
    best_model_classic,
    X=X_test,
    y=y_test_encoded,
    test_data=(X_test, y_test_encoded),
    random_state=RANDOM_STATE,
    n_jobs=-1,
)

pd.DataFrame(results_test_classic)

100%|██████████| 1/1 [00:17<00:00, 17.44s/it, method=XGBClassifier]


,estimator,time,accuracy,precision,recall,f1,conf_mat
0,XGBClassifier,17.312355,1.0,1.0,1.0,1.0,"[[1943.3333333334194, 0.0, 0.0, 0.0, 0.0, 0.0]..."
